In [22]:
import pandas as pd
import re
import multiprocessing

In [23]:
cpa_train_gt = pd.read_csv('SOTAB CPA/sotab_v2_cpa_training_set.csv')
cpa_test_gt = pd.read_csv('SOTAB CPA/sotab_v2_cpa_test_set.csv')

In [24]:
cpa_train_gt

,table_name,main_column_index,column_index,label
0,Book_11x17.pt_September2020_CPA.json.gz,0,3,datePublished
1,Book_11x17.pt_September2020_CPA.json.gz,0,1,isbn
2,Book_11x17.pt_September2020_CPA.json.gz,0,2,numberOfPages
3,Book_1jour-1jeu.com_September2020_CPA.json.gz,0,9,worstRating
4,Book_1jour-1jeu.com_September2020_CPA.json.gz,0,5,datePublished
...,...,...,...,...
109989,TVEpisode_yifytvseries.com_September2020_CPA.j...,0,3,datePublished
109990,TVEpisode_zazangels.com_September2020_CPA.json.gz,0,5,duration
109991,TVEpisode_zazangels.com_September2020_CPA.json.gz,0,1,description
109992,TVEpisode_zazangels.com_September2020_CPA.json.gz,0,2,url


In [25]:
cpa_test_gt

,table_name,main_column_index,column_index,label
0,Book_9facts.co.uk_September2020_CPA.json.gz,0,4,image
1,Book_acequiabooksellers.com_September2020_CPA....,0,4,publisher
2,Book_antipodean.com_September2020_CPA.json.gz,0,4,publisher
3,Book_artisanbooksandbindery.com_September2020_...,0,4,publisher
4,Book_audiofilemagazine.com_September2020_CPA.j...,0,3,genre
...,...,...,...,...
2335,Person_cheshire-live.co.uk_September2020_CPA.j...,0,5,email
2336,Event_bransonshows.com_September2020_CPA.json.gz,0,12,organizer
2337,Event_bransonshows.com_September2020_CPA.json.gz,0,4,eventStatus
2338,MusicAlbum_kkbox.com_September2020_CPA.json.gz,0,2,duration


In [26]:
cpa_test_gt["class"] = cpa_test_gt["table_name"].apply(lambda x: x.split("_")[0])

In [27]:
cpa_test_gt

,table_name,main_column_index,column_index,label,class
0,Book_9facts.co.uk_September2020_CPA.json.gz,0,4,image,Book
1,Book_acequiabooksellers.com_September2020_CPA....,0,4,publisher,Book
2,Book_antipodean.com_September2020_CPA.json.gz,0,4,publisher,Book
3,Book_artisanbooksandbindery.com_September2020_...,0,4,publisher,Book
4,Book_audiofilemagazine.com_September2020_CPA.j...,0,3,genre,Book
...,...,...,...,...,...
2335,Person_cheshire-live.co.uk_September2020_CPA.j...,0,5,email,Person
2336,Event_bransonshows.com_September2020_CPA.json.gz,0,12,organizer,Event
2337,Event_bransonshows.com_September2020_CPA.json.gz,0,4,eventStatus,Event
2338,MusicAlbum_kkbox.com_September2020_CPA.json.gz,0,2,duration,MusicAlbum


In [28]:
gt = {'train':{}, 'test':{}}
for index, row in cpa_train_gt.iterrows():
    if row["table_name"] not in gt['train']:
        gt['train'][row["table_name"]] = {}
    gt['train'][row["table_name"]][row["column_index"]] = row["label"]
    
for index, row in cpa_test_gt.iterrows():
    if row["table_name"] not in gt['test']:
        gt['test'][row["table_name"]] = {}
    gt['test'][row["table_name"]][row["column_index"]] = row["label"]

In [29]:
#Simple Preprocessing

def clean_text(text):
        
    if(isinstance(text, dict)):
        text = ' '.join([ clean_text(v) for k, v in text.items()] )
    elif(isinstance(text, list)):
        text = map(clean_text, text)
        text = ' '.join(text)
        
    if pd.isnull(text):
        return ''
    
    #Remove excess whitespaces
    text = re.sub(' +', ' ', str(text)).strip()
    
    return text

In [30]:
# Prepare format of input datasets for LM models: table_id, [labels], data, label_ids

#ORIGINAL
""" 
def get_all_table_columns(file_name, index):
    
    #By column
    if file_name in cpa_train_gt["table_name"].tolist():
        path = 'SOTAB CPA/Train/'+file_name
    else:
        path = 'SOTAB CPA/Test/'+file_name
    
    df = pd.read_json(path, compression='gzip', lines=True)
        
    cleaned_rows = []
    
    #Main column

    cleaned_main = " ".join([" ".join(clean_text(row).split()[:20]) for row in df[0].tolist()[:5]]) #select 20 words
    
    cleaned_rows.append(cleaned_main)
    
    for row in df.iloc[:, index].tolist():
        cleaned = " ".join(clean_text(row).split()[:20]) #select 20 words
        if cleaned != "":
            cleaned_rows.append(cleaned)
    
    return " ".join(cleaned_rows[:5]) #select rows """

' \ndef get_all_table_columns(file_name, index):\n    \n    #By column\n    if file_name in cpa_train_gt["table_name"].tolist():\n        path = \'SOTAB CPA/Train/\'+file_name\n    else:\n        path = \'SOTAB CPA/Test/\'+file_name\n    \n    df = pd.read_json(path, compression=\'gzip\', lines=True)\n        \n    cleaned_rows = []\n    \n    #Main column\n\n    cleaned_main = " ".join([" ".join(clean_text(row).split()[:20]) for row in df[0].tolist()[:5]]) #select 20 words\n    \n    cleaned_rows.append(cleaned_main)\n    \n    for row in df.iloc[:, index].tolist():\n        cleaned = " ".join(clean_text(row).split()[:20]) #select 20 words\n        if cleaned != "":\n            cleaned_rows.append(cleaned)\n    \n    return " ".join(cleaned_rows[:5]) #select rows '

In [31]:
# Prepare format of input datasets for LM models: table_id, [labels], data, label_ids


#COLUMN PROPERTY MODIFIED

def get_all_table_columns(file_name, index):
    
    #By column
    if file_name in cpa_train_gt["table_name"].tolist():
        path = 'SOTAB CPA/Train/'+file_name
    else:
        path = 'SOTAB CPA/Test/'+file_name
    
    df = pd.read_json(path, compression='gzip', lines=True)
        
    cleaned_rows = []
    
    #Main column
    cleaned_main="{}{}"

    x = " ".join([" ".join(clean_text(row).split()[:20]) for row in df[0].tolist()[:5]]) #select 20 words
    cleaned_main=cleaned_main.format("Column 1: ", x)
    cleaned_rows.append(cleaned_main)

    cleaned_rows.append("\n Column 2:")
    
    
    for row in df.iloc[:, index].tolist():
        get_values_var=7
        cleaned = " ".join(clean_text(row).split()[:20]) #select 20 words
        
        if cleaned != "":
            cleaned_rows.append(cleaned)
        else:
            get_values_var=get_values_var-1

       
    
    return " ".join(cleaned_rows[:get_values_var]) #select rows

In [32]:
test_examples = []
for table in gt['test']:
    for column in gt['test'][table]:
        col_str = get_all_table_columns(table, column)
        test_examples.append([table, column, col_str, gt['test'][table][column], table.split("_")[0]])

In [33]:
col_str

"Column 1: Veal with Chef Rene Rodriguez A King Who Liked to Sing! The Other Half World Record Dancing Ranger Tom and the Musha Rangers \n Column 2: Cook Like a Chef The Wiggles' World The Luminaries The Wiggles' World Tree Fu Tom"

In [34]:
test_examples

[['Book_9facts.co.uk_September2020_CPA.json.gz',
  4,
  "Column 1: Medusa’s Master Angela's Ashes A Baby for Mommy A Handbook for Morning Time The Intentional Brain \n Column 2: http://9facts.co.uk/wp-content/upload/2017/7a7f20ef68c37779ee3ba37cfcca3949ic89f099cfa1e68de4ca26b27af89560egr-assetsc89f099cfa1e68de4ca26b27af89560ecom/images/S/compressedc89f099cfa1e68de4ca26b27af89560ephotoc89f099cfa1e68de4ca26b27af89560egoodreadsc89f099cfa1e68de4ca26b27af89560ecom/books/1328018712l/6579064c89f099cfa1e68de4ca26b27af89560ejpg.jpg http://9facts.co.uk/wp-content/upload/2017/7a7f20ef68c37779ee3ba37cfcca3949ic89f099cfa1e68de4ca26b27af89560egr-assetsc89f099cfa1e68de4ca26b27af89560ecom/images/S/compressedc89f099cfa1e68de4ca26b27af89560ephotoc89f099cfa1e68de4ca26b27af89560egoodreadsc89f099cfa1e68de4ca26b27af89560ecom/books/1400859320l/12831442c89f099cfa1e68de4ca26b27af89560ejpg.jpg http://9facts.co.uk/wp-content/upload/2017/7a7f20ef68c37779ee3ba37cfcca3949ic89f099cfa1e68de4ca26b27af89560egr-assetsc8

In [35]:
cpa_test_gt[cpa_test_gt["table_name"] == "Book_9facts.co.uk_September2020_CPA.json.gz"]

,table_name,main_column_index,column_index,label,class
0,Book_9facts.co.uk_September2020_CPA.json.gz,0,4,image,Book
472,Book_9facts.co.uk_September2020_CPA.json.gz,0,10,inLanguage,Book
733,Book_9facts.co.uk_September2020_CPA.json.gz,0,3,bookFormat,Book
734,Book_9facts.co.uk_September2020_CPA.json.gz,0,5,numberOfPages,Book
735,Book_9facts.co.uk_September2020_CPA.json.gz,0,6,isbn,Book


In [36]:
train_examples = []
for table in gt['train']:
    for column in gt['train'][table]:
        col_str = get_all_table_columns(table, column)
        train_examples.append([table, column, col_str, gt['train'][table][column], table.split("_")[0]])

In [37]:
len(train_examples)

109994

In [38]:
import pickle

In [39]:
file_name='SOTAB CPA/sotabv2-cpa-train-column.pkl'
f = open(file_name,'wb')
pickle.dump(train_examples,f)
f.close()

In [40]:
file_name='SOTAB CPA/sotabv2-cpa-test-column.pkl'
f = open(file_name,'wb')
pickle.dump(test_examples,f)
f.close()